In [5]:
%pip install transformers

import os
import pandas as pd
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments


Note: you may need to restart the kernel to use updated packages.


In [6]:
# Load the model and tokenizer
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

print("DistilGPT-2 loaded successfully!")


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

DistilGPT-2 loaded successfully!


In [18]:
import random

# Define categories with variations
categories = {
    "Content Clarity": [
        "Write a comment about the clarity of an educational video.",
        "Describe how clear and understandable the content of an educational video was.",
        "Provide feedback on whether the video content was easy to follow."
    ],
    "Engagement": [
        "Write a comment about how engaging an educational video was.",
        "Describe if the video was interesting and held your attention.",
        "Provide feedback on whether the educational video kept you engaged."
    ],
    "Relevance": [
        "Write a comment about how relevant the content of an educational video was to a learner.",
        "Provide feedback on whether the video was aligned with the learner's goals.",
        "Describe if the video content was useful and relevant."
    ],
    "Pacing": [
        "Write a comment about whether the pacing of an educational video was too fast, too slow, or just right.",
        "Provide feedback on how well-paced the educational video was.",
        "Describe if the speed of the video content was appropriate for learning."
    ],
    "Technical Issues": [
        "Write a comment about any technical issues in an educational video, such as poor audio or video quality.",
        "Describe if there were any technical problems with the video.",
        "Provide feedback on whether the video had clear audio and visuals."
    ]
}

# Set the pad token to eos_token
tokenizer.pad_token = tokenizer.eos_token

def generate_comments(prompt, num_comments=10):
    # Tokenize input with padding and create attention masks
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    # Generate text
    outputs = model.generate(
        inputs.input_ids,
        attention_mask=inputs.attention_mask,
        max_length=50,
        num_return_sequences=num_comments,  # Generate multiple comments
        do_sample=True,  # Sampling for diverse output
        top_k=40,  # Limit to top-k tokens
        top_p=0.9,  # Nucleus sampling
        temperature=0.8,  # Adjust randomness
        pad_token_id=tokenizer.pad_token_id  # Set pad token ID to padding token
    )
    # Decode the outputs
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Generate synthetic feedback
synthetic_feedback = []
comments_per_category = 3000

for category, prompts in categories.items():
    print(f"Generating comments for category: {category}")
    for _ in range(comments_per_category // 10):  # Generate in batches of 10
        prompt = random.choice(prompts)  # Use a random prompt variation
        comments = generate_comments(prompt, num_comments=10)
        for comment in comments:
            synthetic_feedback.append({"feedback": comment, "category": category})


Generating comments for category: Content Clarity
Generating comments for category: Engagement
Generating comments for category: Relevance
Generating comments for category: Pacing
Generating comments for category: Technical Issues


In [23]:
import pandas as pd
import os

# Load the JSON file
json_file_path = "feedback_comments.json"  # Replace with your file path
df = pd.read_json(json_file_path, lines=True)

# Display the DataFrame
print(df.head())

ValueError: Expected object or value

In [ ]:
import pandas as pd
import os

json_file_path = 'feedback_comments.json'

# Check if the file exists
if not os.path.exists(json_file_path):
    raise FileNotFoundError(f"The file at {json_file_path} does not exist.")

# Check if the file is not empty
if os.path.getsize(json_file_path) == 0:
    raise ValueError(f"The file at {json_file_path} is empty.")

# Read and print the content of the file
with open(json_file_path, 'r') as file:
    content = file.read()
    # print("File content:")
    # print(content)

try:
    df = pd.read_json(json_file_path)
except ValueError as e:
    raise ValueError(f"Error reading JSON file: {e}")

print(df.head())

File content:
[
    {"Feedback": "The explanation was clear and easy to follow.", "Category": "Content Clarity"},
    {"Feedback": "I understood the topic much better after watching this video.", "Category": "Content Clarity"},
    {"Feedback": "The concepts were well-explained, with simple language.", "Category": "Content Clarity"},
    {"Feedback": "The speaker broke down complex ideas into manageable parts.", "Category": "Content Clarity"},
    {"Feedback": "I appreciate how the examples made the content more understandable.", "Category": "Content Clarity"},
    {"Feedback": "The clarity of the instructions was excellent throughout the video.", "Category": "Content Clarity"},
    {"Feedback": "It was easy to grasp the main points, even for a beginner like me.", "Category": "Content Clarity"},
    {"Feedback": "The video avoided unnecessary jargon, which I liked.", "Category": "Content Clarity"},
    {"Feedback": "The diagrams and visuals helped clarify the content a lot.", "Category